In [105]:
import numpy as np
def load_spiral():
    data = np.loadtxt("dataset/spiral.txt")
    x = data[:, 0:2]
    t_raw = data[:, 2:3]
    t = np.zeros_like(data)
    for i in range(len(data)):
        t[i][int(t_raw[i][0] - 1)] = 1
    x = np.float32(x)
    t = np.int8(t)
    t_raw = np.int8(t_raw) -1
    return x, t

In [106]:
class Sigmoid:
    def __init__(self):
        self.params = []
        
    def forward(self, x):
        return 1 / (1 + np.exp(-x))
    
class Affine:
    def __init__(self, W, b):
        self.params = [W, b]
        
    def forward(self, x):
        W, b = self.params
        out = np.dot(x, W) + b
        return out

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size):
        I, H, O = input_size, hidden_size, output_size
        
        # 重みの初期化
        W1 = 0.01 * np.random.randn(I, H)
        b1 = np.zeros(H)
        W2 = 0.01 * np.random.randn(H, O)
        b2 = np.zeros(O)
        
        # レイヤの生成
        self.layers  = [
            Affine(W1, b1),
            Sigmoid(),
            Affine(W2, b2)
        ]
        self.loss_layer = SoftmaxWithLoss()
        
        # すべての重みと勾配を配列にまとめる
        self.params, self.grads = [], []
        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads
    
    def predict(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x
    
    def forward(self, x, t):
        score = self.predict(x)
        loss = self.loss_layer.forward(score, t)
        return loss
    
    def backward(self, dout=1):
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers):
            dout = layer.backward(dout)

In [116]:
class MatMul:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.x = None
    
    def forward(self, x):
        W, = self.params
        out = np.dot(x,  W)
        self.x = x
        return out
    
    def backward(self, dout):
        W, = self.params
        dx = np.dot(dout, W.T)
        dW = np.dot(self.x.T, dout)
        self.grads[0][...] = dW
        return dx
    
class Sigmoid:
    def __init__(self):
        self.params, self.grads = [], []
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

class Affine:
    def __init__(self, W, b):
        self.params = [W, b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.x = None
    
    def forward(self, x):
        W, b = self.params
        out = np.dot(x, W) + b
        self.x = x
        return out
    
    def backward(self, dout):
        W, b = self.params
        dx = np.dot(dout, W.T)
        dW = np.dot(self.x.T, dout)
        db = np.sum(dout, axis=0)
        
        self.grads[0][...] = dW
        self.grads[1][...] = db
        return dx

def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y)) / batch_size

def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx

In [117]:
class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr
    
    def update(self, params, grads):
        for i in range(len(params)):
            params[i] -= self.lr * grads[i]

In [123]:
max_epoch = 300
batch_size = 30
hidden_size = 10
learning_rate = 0.1

x, t = load_spiral()
model = TwoLayerNet(input_size=2, hidden_size=hidden_size, output_size=3)
optimizer = SGD(lr=learning_rate)

data_size = len(x)
max_iters = data_size // batch_size
loss_list = []
total_loss = 0
loss_count = 0

for epoch in range(max_epoch):
    idx = np.random.permutation(data_size)
    x = x[idx]
    t = t[idx]
    
    for iters in range(max_iters):
        batch_x = x[iters*batch_size:(iters+1) * batch_size]
        batch_t = t[iters*batch_size:(iters+1) * batch_size]
        
        loss = model.forward(batch_x, batch_t)
        model.backward()
        optimizer.update(model.params, model.grads)
        
        total_loss += loss
        loss_count += 1
        
        if (iters+1) % 10 == 0:
            avg_loss = total_loss / loss_count
            print('| epoch % d |  iter %d  /  %d  |  loss  %.2f' % (epoch+1, iters+1, max_iters, avg_loss))
            loss_list.append(avg_loss)
            total_loss, loss_count = 0, 0

| epoch  1 |  iter 10  /  10  |  loss  4.70
| epoch  2 |  iter 10  /  10  |  loss  5.07
| epoch  3 |  iter 10  /  10  |  loss  5.07
| epoch  4 |  iter 10  /  10  |  loss  4.93
| epoch  5 |  iter 10  /  10  |  loss  5.10
| epoch  6 |  iter 10  /  10  |  loss  5.47
| epoch  7 |  iter 10  /  10  |  loss  5.56
| epoch  8 |  iter 10  /  10  |  loss  6.09
| epoch  9 |  iter 10  /  10  |  loss  5.05
| epoch  10 |  iter 10  /  10  |  loss  6.29
| epoch  11 |  iter 10  /  10  |  loss  6.19
| epoch  12 |  iter 10  /  10  |  loss  5.60
| epoch  13 |  iter 10  /  10  |  loss  5.73
| epoch  14 |  iter 10  /  10  |  loss  6.01
| epoch  15 |  iter 10  /  10  |  loss  5.80
| epoch  16 |  iter 10  /  10  |  loss  5.39
| epoch  17 |  iter 10  /  10  |  loss  5.89
| epoch  18 |  iter 10  /  10  |  loss  5.43
| epoch  19 |  iter 10  /  10  |  loss  5.81
| epoch  20 |  iter 10  /  10  |  loss  5.95
| epoch  21 |  iter 10  /  10  |  loss  5.01
| epoch  22 |  iter 10  /  10  |  loss  5.33
| epoch  23 |  iter

In [104]:
a = np.loadtxt("dataset/spiral.txt")

In [35]:
x = a[:, 0:2].shape
t = a[:, 2:3]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [36]:
b = np.zeros_like(a)

In [39]:
b[0][1] = 1

In [40]:
b

array([[ 0.,  1.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.

In [50]:
data = np.loadtxt("dataset/spiral.txt")
x = data[:, 0:2]
t_raw = data[:, 2:3]
t = np.zeros_like(data)
for i in range(len(data)):
    t[i][int(t_raw[i][0] - 1)] = 1

In [115]:
batch_t[np.arange(30), batch_t]

IndexError: shape mismatch: indexing arrays could not be broadcast together with shapes (30,) (30,3) 

In [120]:
model.grads

[array([[ -1.96931422e-06,  -1.35015159e-06,  -1.96988530e-06,
          -1.13426720e-06,  -1.25350033e-06,  -1.32029090e-06,
          -1.06664948e-06,  -1.30208235e-06,  -7.12881055e-07,
          -1.58535959e-06],
        [ -4.61829950e-07,  -3.58590605e-07,  -4.62239400e-07,
          -3.22711200e-07,  -3.42616525e-07,  -3.53676630e-07,
          -3.12241142e-07,  -3.50547161e-07,  -2.55689591e-07,
          -3.97336299e-07]]),
 array([ -1.12881612e-07,  -7.92910983e-08,  -1.12925891e-07,
         -6.75937109e-08,  -7.40574609e-08,  -7.76749570e-08,
         -6.39657723e-08,  -7.66835928e-08,  -4.49187931e-08,
         -9.20264679e-08]),
 array([[-0.29960085, -0.35046816, -0.31659766],
        [-0.29960085, -0.35046816, -0.31659766],
        [-0.29960085, -0.35046816, -0.31659766],
        [-0.29960085, -0.35046816, -0.31659766],
        [-0.29960085, -0.35046816, -0.31659766],
        [-0.29960085, -0.35046816, -0.31659766],
        [-0.29960085, -0.35046816, -0.31659766],
       